# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
#Here I stablish a path to the cities.csv file
herepath = os.path.dirname(os.path.abspath('__file__'))
weather_data = os.path.join(herepath,'output_data/cities.csv')

#Then we transformed it into a Pandas Data Frame
Cities_weather = pd.read_csv(weather_data)
Cities_weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
#Here we define the asked parameters as variables
locations = Cities_weather[["Lat", "Lng"]]
humidity = Cities_weather['Humidity']
max_int = Cities_weather['Humidity'].max()

#Then we create a gmaps figure
fig = gmaps.figure()
#And here we use the tool 'heatmap_layer' from gmaps to create a heatmap layer
humidity_heat_map = gmaps.heatmap_layer(locations, weights = humidity, max_intensity = max_int, point_radius = 20)
#And then we add this layer to the previos figure we create
fig.add_layer(humidity_heat_map)
#We show the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
#Here we uso loc function to select the data we desire, with cloudiness less than ten, wind speed less than 5 and Temperature between 21 to 25
Ideal_weather = Cities_weather.loc[((Cities_weather['Cloudiness'] < 10) & (Cities_weather['Wind Speed'] < 5) & (Cities_weather['Max Temp'].between(21,25)))]

#Ideal_weather.count()    
#There's no null values :)
Ideal_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
422,422,susuman,0,RU,1558378875,75,62.78,148.15,22.38,3.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
#Store into variable named hotel_df.
hotel_df = pd.DataFrame(Ideal_weather)
#Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Location
422,422,susuman,0,RU,1558378875,75,62.78,148.15,22.38,3.15,,


In [34]:
#Set parameters to search for hotels with 5000 meters.
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
area = 5000
kind = 'hotel'
key = g_key

#We create a params dictionary in order to respond the api parameters that google asks
#The only parameter who is missed is location, but we will add further while we iterate in the df
params = { 
    'radius': area,
    'type' : kind,
    'key': key}

hotel_df['Hotel Location'] = ""

#Hit the Google Places API for each city's coordinates.
#First we iterate trhoug different cordenaste to create the differente api calls
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']
    city = row['City']
    country = row['Country']
    #Here we sort cordenates as google documentation ask 
    cordenates = f'{lat},{lon}'
    #And add them to the params dictionary we create
    params['location'] = cordenates
    #Create a response variable to store our request
    response = requests.get(url, params=params).json()
    #As google documentations says, the data we look for are into a 'results' list in a dictionary so we grab just that
    results = response['results']
    
    #If there is a registered hotel in the city we take the data of the closest and add it to our hotel_df
    try:
        print(f"Closest hotel to {city},{country} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Aparently there's no hotels here... skipping.")
        pass
    print("------------")

Aparently there's no hotels here... skipping.
------------


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [33]:
marker = gmaps.marker_layer(locations2, info_box_content= hotel_info)
# Add marker layer ontop of heat map
fig.add_layer(marker)

# Display Map
fig

NameError: name 'locations2' is not defined